In [ ]:
!pip install jsonlines

In [ ]:
%cd data/beauty
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm

/content/drive/MyDrive/LLM_ESR_2023_v3/data/Books


In [ ]:

data = json.load(open("handled/item2attributes.json", "r"))
review = json.load(open("handled/items2rep_sentences.json", "r"))

In [ ]:
import json
with open("/handled/items2rep_sentences.json", "r", encoding="utf-8") as f:
    id_map_item = json.load(f)

print("total item:", len(id_map_item))

for i, (k, v) in enumerate(id_map_item.items()):
    print(f"{i} ▶ key={k} (type={type(k)}), value sample={str(v)[:200]}")
    if i >= 5:  # 5개까지만
        break


총 아이템 수: 496442
0 ▶ key=1 (type=<class 'str'>), value sample=['[\'Hugh Laurie’s sarcastic humor is totally enjoyable, but the story can get lost in it.\'\n \'As the main character in the book, it was less than interesting.\'\n \'<br />I really wanted to like th
1 ▶ key=2 (type=<class 'str'>), value sample=['[\'Absolutely entertaining, as are all Stephanie Plum novels.\'\n "I can\'t wait for the next book."\n \'That Janet Evanovich can keep the dynamic tension between Morelli and Ranger alive for so man
2 ▶ key=3 (type=<class 'str'>), value sample=['[\'I must save my five-star ratings for top-notch literature, but while the plot of this book is a bit preposterous, it is engrossing and really funny.\'\n "The first of the &#34;Dortmunder&#34; adv
3 ▶ key=4 (type=<class 'str'>), value sample=['[\'The recipes are fun and unique.\'\n "As I am just beginning to delve into Korean cooking at my DH\'s request, I have found this cookbook very helpful."\n \'I remember looking through this cookboo


In [ ]:
import json, os
P = "handled"

# 메타: {item_id(str): ...}
with open(os.path.join(P, f"item2attributes.json"), "r", encoding="utf-8") as f:
    item2attributes = {str(k): v for k, v in json.load(f).items()}

# aspects: {item_id(str): [scores]}
with open(os.path.join(P, f"items2rep_sentences.json"), "r", encoding="utf-8") as f:
    items2aspect = {str(k): v for k, v in json.load(f).items()}

ids_attr = set(item2attributes.keys())
ids_asp  = set(items2aspect.keys())

print("meta item:", len(ids_attr))
print("except item":", len(ids_asp))
print("ids_asp:", len(ids_attr & ids_asp))
print("only meta:", len(ids_attr - ids_asp))
print("only except:", len(ids_asp - ids_attr))


메타 아이템 수: 496442
앱섹트 아이템 수: 496442
교집합: 496442
메타만 있는 수: 0
앱섹트만 있는 수: 0


In [ ]:
items2aspect['266']
item2attributes['266']

[1, 14, 15]

In [ ]:


prompt_template = "The item has following attributes:\n name is <TITLE>; brand is <BRAND>; price is <PRICE>.\n"
feat_template   = "The item has following features: <CATEGORIES>.\n"
desc_template   = "The item has following descriptions: <DESCRIPTION>.\n"
review_template = "The item has following reviews : <REVIEW>.\n"

def get_attri(s, key, meta):
    if isinstance(meta, list):
        return s.replace(f"<{key.upper()}>", "; ".join(map(str, meta)))
    return s.replace(f"<{key.upper()}>", str(meta.get(key, "unknown")) if isinstance(meta, dict) else "unknown")
def get_aspects(s, key, meta):
    if isinstance(meta, list):
        return s.replace(f"<{key.upper()}>", "; ".join(map(str, meta)))

def get_feat(s, key, meta):
    if isinstance(meta, list):
        return s.replace(f"<{key.upper()}>", "; ".join(map(str, meta)))
    cats = meta.get(key) if isinstance(meta, dict) else None
    if not cats:
        return s
    cats = "; ".join(cats[0]) if isinstance(cats, list) and cats and isinstance(cats[0], list) else "; ".join(cats)
    out = s.replace(f"<{key.upper()}>", cats)
    return out[:2048] if len(out) > 2048 else out

item_data = {}
for iid in sorted(ids_attr & ids_asp, key=lambda x:int(x)):
    meta = item2attributes.get(iid, {})
    aspv = items2aspect.get(iid, None)

    s  = get_attri(prompt_template, "title", meta)
    s  = get_attri(s, "brand", meta)
    s  = get_attri(s, "price", meta)
    s2 = get_feat(feat_template, "categories", meta)
    s3 = get_attri(desc_template, "description", meta)
    s4 = review_template.replace("<REVIEW>", str(aspv))
    item_data[iid] = s + s2 + s3
    # item_data[iid] = s + s2 + s3 + s4


In [ ]:
item_data['53']

'The item has following attributes:\n name is 1; 2; 3; brand is 1; 2; 3; price is 1; 2; 3.\nThe item has following features: 1; 2; 3.\nThe item has following descriptions: 1; 2; 3.\n'

In [ ]:
with open("handled/item_str.json", "w") as f:
    json.dump(item_data, f)

In [ ]:
item_data = json.load(open("handled/item_str.json", "r"))

In [ ]:
import jsonlines

def save_data(data_path, data):
    '''write all_data list to a new jsonl'''
    with jsonlines.open("handled/"+ data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

id_map = json.load(open("handled/id_map.json", "r"))["item2id"]
json_data = []
for key, value in item_data.items():
    json_data.append({
        "input": value,
        "target": "",
        "item": key,
        "item_id": int(key)   # 그냥 int로 변환
    })


save_data("item_str.jsonline", json_data)


In [ ]:
%env OPENAI_API_KEY= your api key

In [ ]:
import os, requests, json

url = "https://api.openai.com/v1/embeddings"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # 환경변수에서 불러오기

payload = {
   "model": "text-embedding-ada-002",   # 최신은 text-embedding-3-small 권장
   "input": "The food was delicious and the waiter..."
}
headers = {
   'Authorization': f'Bearer {OPENAI_API_KEY}',
   'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())


200
{'object': 'list', 'data': [{'object': 'embedding', 'index': 0, 'embedding': [0.0022756963, -0.009305916, 0.015742613, -0.0077253063, -0.0047450014, 0.014917395, -0.009807394, -0.038264707, -0.0069127847, -0.028590616, 0.025251659, 0.018116701, -0.0036309576, -0.02554366, 0.00055543496, -0.016428178, 0.02828592, 0.0054083494, 0.009610611, -0.016415482, -0.015412526, 0.004272088, 0.0069953064, -0.007223828, -0.0039007403, 0.018573744, 0.008734611, -0.022699833, 0.011508612, 0.023893224, 0.015602961, -0.0035706533, -0.034963835, -0.0041514793, -0.026178442, -0.02150644, -0.0057066972, 0.011768873, 0.008455306, 0.004129262, 0.019157745, -0.014358787, 0.008982176, 0.0063605234, -0.04570436, 0.017900875, -0.005570219, -0.0007716578, -0.02215392, -0.0039229575, 0.02101131, -0.017608874, -0.011699047, -0.02256018, 0.01633931, 0.017164527, -0.00838548, 0.0015901309, 0.02507392, -0.024997747, 0.007807828, 0.005798741, -0.022115832, 0.002948566, -0.0061764363, -0.025556356, -0.008074437, 0.0

In [ ]:
def get_response(prompt):
    url = "https://api.openai.com/v1/embeddings"

    payload = json.dumps({
    "model": "text-embedding-ada-002",
    "input": prompt
    })
    headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    re_json = json.loads(response.text)

    return re_json["data"][0]["embedding"]

In [ ]:
item_emb = {}

In [ ]:
value_list = []
count = 0
for key, value in tqdm(item_data.items()):
    if len(value) > 4096:
        value_list.append(key)
    else:
      count +=1

100%|██████████| 496442/496442 [00:00<00:00, 2239755.12it/s]


In [ ]:
print(len(item_data))

496442


In [ ]:
CHUNK = 10000
keys = list(item_data.keys())
chunks_list = []


for start in range(0, len(keys), CHUNK):
    sub_keys = keys[start:start + CHUNK]
    sub_dict = {k: item_data[k] for k in sub_keys}
    chunks_list.append(sub_dict)


for i, chunk in enumerate(chunks_list):

    globals()[f'chunks{i}'] = chunk

print(f"데이터를 총 {len(chunks_list)}개의 청크(chunks0 ~ chunks{len(chunks_list)-1})로 분할하고 동적 할당 완료.")

데이터를 총 50개의 청크(chunks0 ~ chunks49)로 분할하고 동적 할당 완료.


In [ ]:
import pickle
from datetime import datetime

In [ ]:

import os, time, pickle, random
from typing import Dict, List
from tqdm import tqdm

# OpenAI SDK v1
from openai import OpenAI
client = OpenAI()

MODEL = "text-embedding-3-small"
BATCH_SIZE = 100
MAX_LEN = 15000
OUT_DIR = "handled"
os.makedirs(OUT_DIR, exist_ok=True)

def _trim(txt: str, max_len: int = MAX_LEN) -> str:
    if txt is None:
        return ""
    return txt[:max_len-1] if len(txt) > max_len else txt

def _retry_delay(attempt: int) -> float:

    base = min(60, 2 ** attempt)   # 최대 60초
    return base * (0.5 + random.random() * 0.5)

def embed_items_in_batches(item_data: Dict[str, str], out_path: str):
    """
    item_data: {item_id: text}
    out_path : 'handled/item_embX.pkl'
    """

    item_emb = {}
    if os.path.exists(out_path):
        try:
            with open(out_path, "rb") as f:
                item_emb = pickle.load(f)
            print(f"[resume] loaded {len(item_emb)} embeddings from {out_path}")
        except Exception as e:
            print("[resume] failed to load existing file:", e)


    keys: List[str] = []
    texts: List[str] = []
    for k, v in item_data.items():
        if k in item_emb:
            continue
        keys.append(k)
        texts.append(_trim(v))

    if not keys:
        print("[done] nothing to do; all embeddings already exist.")
        return

    total = len(keys)
    print(f"[start] to-embed: {total}, batch_size={BATCH_SIZE}")

    pbar = tqdm(total=total, desc="Embedding(items)", unit="item")

    for start in range(0, total, BATCH_SIZE):
        end = min(start + BATCH_SIZE, total)
        batch_keys = keys[start:end]
        batch_texts = texts[start:end]


        attempt = 0
        while True:
            try:
                resp = client.embeddings.create(
                    model=MODEL,
                    input=batch_texts
                )

                for k, item in zip(batch_keys, resp.data):
                    item_emb[k] = item.embedding
                pbar.update(len(batch_keys))
                break
            except Exception as e:
                attempt += 1
                delay = _retry_delay(attempt)
                print(f"[warn] batch {start}-{end} failed (attempt {attempt}): {e}")
                print(f"       retry in {delay:.1f}s")
                time.sleep(delay)
                if attempt >= 6:
                    print("[error] giving up this batch; partial will be saved.")
                    break


        if (start // BATCH_SIZE) % 5 == 0:
            try:
                with open(out_path, "wb") as f:
                    pickle.dump(item_emb, f)
            except Exception as e:
                print("[checkpoint] save failed:", e)

    pbar.close()

    with open(out_path, "wb") as f:
        pickle.dump(item_emb, f)
        f.flush()            # 버퍼 비우기
        os.fsync(f.fileno()) # OS 레벨에서 디스크 쓰기 강제
    print(f"[done] saved {len(item_emb)} embeddings -> {out_path}")
    time.sleep(3)


for i in range(0,100):
  if i == 15:
    var = f"chunks{i}"
    if var not in globals():
        print(f"[skip] {var} not found in globals()")
        continue
    out_path = os.path.join(OUT_DIR, f"item_emb{i}.pkl")
    print(f"\n=== Part {i}: {var} -> {out_path} ===")
    embed_items_in_batches(globals()[var], out_path)






=== Part 15: chunks15 -> handled/item_emb15.pkl ===
[start] to-embed: 10000, batch_size=100


Embedding(items): 100%|██████████| 10000/10000 [02:47<00:00, 59.77item/s] 


[done] saved 10000 embeddings -> handled/item_emb15.pkl


In [ ]:

import json, pickle, numpy as np
id_map = json.load(open("handled/id_map.json", "r"))["id2item"]
item_emb = pickle.load(open("handled/item_emb.pkl", "rb"))

emb_list = []
for idx in range(1, len(item_emb) + 1):
    meta_emb = item_emb[str(idx)]  # or item_emb[idx] if int key
    emb_list.append(meta_emb)

emb_list = np.array(emb_list)
pickle.dump(emb_list, open("handled/itm_emb_np.pkl", "wb"))


/content/drive/MyDrive/LLM_ESR_2023_v3/data/Books
